In [1]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn import impute
import itertools as it
import time as time
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor


data = pd.read_csv('train.csv')

#Create cluster dummies
from sklearn.cluster import KMeans, DBSCAN
# clusters = KMeans(random_state=0, n_clusters = 10)
# clusters.fit(data)

# pred = clusters.predict(data)
# data['Cluster'] = pred
# data.Cluster = data.Cluster.astype(str)

# Log transformation
y = np.log(data.y)
X = data.drop('y', axis=1)


# Drop columns with >5% NaN
inst = X.shape[0]
nan_col = [col for col in X if (X[col].isna().sum() / inst > 0.05)]
X = X.drop(nan_col, axis=1)


#Impute NaN using KNNImputer
imputer = impute.KNNImputer(n_neighbors=5)
X = imputer.fit_transform(X)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X)

In [2]:
# Delete 0 variance columns
X = pd.DataFrame(X)
zero_var = []

print(X.shape)

res=[]
for col in X:
     if len(X[col].unique()) == 1:
        res.append(col)
        X = X.drop(col, axis=1)

print('Columns dropped:', len(res))
# 11 columns with only 1 value

print(X.shape)

(5380, 727)
Columns dropped: 11
(5380, 716)


In [ ]:
# # # Remove Correlated Variables
# def remove_collinear_features(x, threshold):
#     # Calculate the correlation matrix
#     corr_matrix = x.corr()
#     iters = range(len(corr_matrix.columns) - 1)
#     drop_cols = []

#     # Iterate through the correlation matrix and compare correlations
#     for i in iters:
#         for j in range(i+1):
#             item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
#             col = item.columns
#             row = item.index
#             val = abs(item.values)

#             # If correlation exceeds the threshold
#             if val >= threshold:
#                 # Print the correlated features and the correlation value
#                 #print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
#                 drop_cols.append(col.values[0])

#     # Drop one of each pair of correlated columns
#     drops = set(drop_cols)
#     x = x.drop(columns=drops)
#     print('Removed Columns {}'.format(drops))
#     return x, drops

# X, drops = remove_collinear_features(X, 0.8)
# print(len(drops))

Removed Columns {8, 10, 11, 12, 21, 29, 30, 33, 34, 37, 38, 39, 40, 42, 44, 45, 50, 52, 53, 54, 55, 56, 57, 61, 62, 63, 64, 66, 68, 70, 75, 77, 79, 80, 83, 84, 86, 89, 90, 92, 94, 96, 97, 99, 101, 102, 103, 104, 107, 108, 110, 111, 113, 115, 117, 118, 120, 121, 124, 125, 128, 131, 133, 134, 135, 138, 139, 140, 141, 142, 143, 145, 146, 149, 150, 151, 152, 153, 154, 155, 156, 158, 160, 162, 164, 165, 166, 168, 169, 170, 173, 175, 176, 177, 178, 181, 182, 183, 186, 187, 188, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 243, 244, 246, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 265, 266, 267, 268, 269, 270, 272, 273, 274, 277, 278, 279, 280, 281, 282, 283, 284, 286, 287, 288, 290, 291, 292, 293, 295, 297, 299, 300, 301, 302, 304, 305, 307, 308, 310, 311, 312, 315, 316, 317

In [3]:
#Feature Selection

#Set a random column to assist search for non-important columns
random = pd.Series(np.random.randn(X.shape[0]))
X['random'] = random

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# params = {'n_estimators': [400],
#           'max_depth': [5,15,25],
#           'max_leaf_nodes':[500,1000,1500],
#           'max_features': [0.3, 0.6, 0.9]}

# cv = KFold(n_splits=5, shuffle=True, random_state=1)
# grid = GridSearchCV(RandomForestRegressor(), n_jobs=-1, param_grid=params, cv=cv, scoring='neg_mean_squared_error')
# grid.fit(X, y)

X.columns = X.columns.astype(str)
rf = RandomForestRegressor().fit(X,y)

imp = rf.feature_importances_

In [4]:
imp = rf.feature_importances_
importances = pd.DataFrame({'feature':X.columns, 'importance':imp})

random_imp = importances.loc[191, 'importance']
selected_predictors = importances[importances['importance'] > random_imp]['feature'].astype(str).tolist()
selected_predictors.remove('random')

#Down to 36 predictors after importance selection
X = X.loc[:,selected_predictors]

In [ ]:
len(selected_predictors)

670

In [ ]:
lasso = Lasso(alpha=0.001) 
lasso.fit(X,y)

lasso_coef = pd.DataFrame({'feature':X.columns, 'importance':lasso.coef_})
random_coef = lasso_coef.loc[191, 'importance']
lasso_selected_predictors = lasso_coef[lasso_coef['importance'] > random_coef]['feature'].astype(str).tolist()


len(lasso_selected_predictors)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.267e+01, tolerance: 5.359e-01
  model = cd_fast.enet_coordinate_descent(


137

In [ ]:
#capturing only linear importance?
selected_predictors = selected_predictors.intersection(lasso_selected_predictors)
selected_predictors = list(selected_predictors)
X = X.loc[: ,pd.Series(selected_predictors).astype(int)]

AttributeError: ignored

In [ ]:
X.shape

(5380, 132)

In [ ]:
# Tuned RandomForest
params = {'max_depth':[20, 21, 22],
          'max_leaf_nodes':[150, 200, 250],
          'bootstrap': [True],
         'max_features': [0.8, 0.9, 1]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)

rf = GridSearchCV(RandomForestRegressor(random_state=1, n_jobs=-1, bootstrap=True, n_estimators=100), 
                                      param_grid =params, cv=cv, n_jobs=-1, verbose=1, scoring = 'neg_mean_squared_error')

rf.fit(X, y)

print('Best Parameters : ',rf.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters :  {'bootstrap': True, 'max_depth': 22, 'max_features': 0.9, 'max_leaf_nodes': 250}


In [33]:
tuned_rf = RandomForestRegressor(random_state=1, n_jobs=-1, bootstrap=True, n_estimators=100,
                                 max_depth= 22, max_features= 0.9, max_leaf_nodes= 250).fit(X,y)
np.sqrt(mean_squared_error(np.exp(y), np.exp(tuned_rf.predict(X))))

7.618544602263436

In [ ]:
tuned_rf = rf.best_estimator_
np.sqrt(mean_squared_error(np.exp(y), np.exp(tuned_rf.predict(X))))

7.83581182987542

In [ ]:
# Lasso -- tune this
np.warnings.filterwarnings('ignore')

score = pd.DataFrame(columns=['a', 'rmse'])
for a in np.arange(0.001, 0.2, 0.001):
    lasso = Lasso(alpha=a)
    lasso.fit(X, y)
    
    score = score.append(pd.DataFrame({'alpha':a, 'rmse':-cross_val_score(lasso, X, y, cv=5, scoring='neg_mean_squared_error').mean()}, index=[0]))

score.loc[score['rmse'].argmin()]

KeyError: ignored

In [ ]:
score.reset_index(drop=True).loc[score['rmse'].argmin()]

a             NaN
rmse     0.534895
alpha       0.005
Name: 4, dtype: object

In [34]:
lasso = Lasso(alpha=0.05).fit(X,y)
np.sqrt(mean_squared_error(np.exp(y), np.exp(lasso.predict(X))))


10.545602641274199

In [5]:
# Bagged MARS
from pyearth import Earth
from sklearn.ensemble import BaggingRegressor

# params = {'max_samples': [.6,.85,1.0],
#           'max_features': [0.5,.75,.85,1.0],
#           'bootstrap': [True, False],
#           'bootstrap_features': [True, False]}

# cv = KFold(n_splits=5,shuffle=True,random_state=1)
earth = BaggingRegressor(Earth(), n_estimators=100, random_state=1, n_jobs=-1).fit(X,y)

# grid_search = RandomizedSearchCV(BaggingRegressor(earth, n_estimators=100) 
#                                   param_distributions=params, ))


In [6]:
train_pred = earth.predict(X)
np.sqrt(mean_squared_error(np.exp(y), np.exp(train_pred)))

9.458935892924227

In [7]:
#Predicting residuals
resid = train_pred - y
earth2 = BaggingRegressor(Earth(), n_estimators=100, random_state=1, n_jobs=-1).fit(X, resid)

In [32]:
pred_resid = earth2.predict(X)
# pred_resid.min()
#Cap residuals??

earth_train_pred = train_pred - pred_resid
np.sqrt(mean_squared_error(np.exp(y), np.exp(final_pred)))

9.322986134976503

In [ ]:
# XGBoost
param_grid = {'max_depth': [4,6,8],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 1, 10],
                'n_estimators':[100, 500, 1000],
                'gamma': [0, 10, 100],
                'subsample': [0.5, 0.75, 1.0],
                'colsample_bytree': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=xgb.XGBRegressor(random_state=1),                                                       
                             param_distributions = param_grid,
                             n_iter=100,
                             verbose = 1,
                             n_jobs=-1,
                             cv = cv,
                             scoring = 'neg_mean_squared_error')
optimal_params.fit(X,y)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Optimal parameter values = {'subsample': 0.75, 'reg_lambda': 10, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 1.0}
Optimal cross validation R-squared =  -0.51388692589406


In [35]:
xg = xgb.XGBRegressor(random_state=1, subsample= 0.75, reg_lambda= 10, n_estimators= 500, 
                      max_depth= 4, learning_rate= 0.05, gamma= 0, colsample_bytree= 1.0).fit(X,y)
np.sqrt(mean_squared_error(np.exp(y), np.exp(xg.predict(X))))

6.013296261036095

In [ ]:
# {'subsample': 0.75, 'reg_lambda': 10, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 1.0}
# xg = optimal_params.best_estimator_
np.sqrt(mean_squared_error(np.exp(y), np.exp(xg.predict(X))))

6.644691089082993

In [ ]:
# Neural Network - need to work on overfitting here
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import tensorflow as tf
# tf.config.run_functions_eagerly(True) #Causing error for me

from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

network = Sequential()
network.add(InputLayer(input_shape=(458,)))
network.add(Dense(1500, activation='linear', kernel_initializer='he_normal'))
network.add(Dropout(0.3))

network.add(Dense(1000, activation='linear', kernel_initializer='he_normal'))
network.add(Dropout(0.3))
network.add(Dense(500, activation='relu', kernel_initializer='he_normal'))
network.add(Dropout(0.3))
network.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
network.add(Dropout(0.3))
network.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
network.add(Dropout(0.3))
network.add(Dense(1, activation='linear', kernel_initializer='he_normal'))


network.compile(loss='mse', optimizer='adam')

history = network.fit(X, y, verbose=0, epochs=50, batch_size=1000, validation_split=0.1,
                      callbacks=[EarlyStopping('val_loss', patience=int(10))])

nn = network

nn_pred = nn.predict(X)
np.sqrt(mean_squared_error(np.exp(y), np.exp(nn_pred)))

169/169 [==============================] - 1s 3ms/step


12.85073601595888

In [ ]:
nn_pred = nn.predict(X)
np.sqrt(mean_squared_error(np.exp(y), np.exp(nn_pred)))


169/169 [==============================] - 1s 3ms/step


12.85073601595888

AttributeError: ignored

In [ ]:
history = network.fit(X, resid, epochs=50, batch_size=1000, validation_split=0.1,
                      callbacks=[EarlyStopping('val_loss', patience=int(10))])
resid_pred = network.predict(resid)

NameError: ignored

In [ ]:
# CatBoost -- Tune this

param_grid = {'max_depth': [4,6,8],
              'num_leaves': [20, 31, 40],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 10, 100],
                'n_estimators':[100, 500, 1000],
                'subsample': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),                                                       
                             param_distributions = param_grid, n_iter = 100,
                             verbose = 1,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X,y)

# np.sqrt(mean_squared_error(np.exp(y), np.exp(model_cat.predict(X))))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                   estimator=<catboost.core.CatBoostRegressor object at 0x7f7a18d4c4c0>,
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [4, 6, 8],
                                        'n_estimators': [100, 500, 1000],
                                        'num_leaves': [20, 31, 40],
                                        'reg_lambda': [0, 10, 100],
                                        'subsample': [0.5, 0.75, 1.0]},
                   random_state=1, verbose=1)

In [36]:
# optimal_params.best_params_
# model_cat = optimal_params.best_estimator_
# {'subsample': 0.75,
#  'reg_lambda': 10,
#  'num_leaves': 31,
#  'n_estimators': 1000,
#  'max_depth': 4,
#  'learning_rate': 0.1}

model_cat = CatBoostRegressor(random_state=1, verbose=False, subsample=0.75, reg_lambda=10, num_leaves=31,
                              n_estimators=1000, max_depth=4, learning_rate=0.1).fit(X,y)
np.sqrt(mean_squared_error(np.exp(y), np.exp(model_cat.predict(X))))


5.602155239054284

In [48]:
#Bagged decision trees?
from sklearn.tree import DecisionTreeRegressor
params = {'base_estimator': [DecisionTreeRegressor(random_state = 1)],#Comparing bagging with a linear regression model as well
          'n_estimators': [100],
          'max_samples': [0.5,1.0],
          'max_features': [0.5,1.0],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), 
                                      param_grid =params, cv=cv, n_jobs=-1, verbose=1)
bagging_regressor_grid.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=BaggingRegressor(n_jobs=-1, random_state=1), n_jobs=-1,
             param_grid={'base_estimator': [DecisionTreeRegressor(random_state=1)],
                         'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 1.0], 'max_samples': [0.5, 1.0],
                         'n_estimators': [100]},
             verbose=1)

In [51]:
print(bagging_regressor_grid.best_params_)
bag = bagging_regressor_grid.best_estimator_
np.sqrt(mean_squared_error(np.exp(y), np.exp(bag.predict(X))))


{'base_estimator': DecisionTreeRegressor(random_state=1), 'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 100}


6.981503097739694

In [73]:
# ensemble
#Stacking using LinearRegression as the metamodel
#, ('rf', tuned_rf)
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
#,('lasso', lasso), ('rf', tuned_rf), ('bag',bag)
en = StackingRegressor(estimators = [('cat', model_cat), ('xg', xg), ('lasso', lasso), ('rf', tuned_rf)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X,y)

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x7f7fc8852d40>),
                              ('xg',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_t...
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=1, ...)),
                              ('lasso', Lasso(alpha=0.05))],
                  final_estimator=LinearRegression())

In [74]:
np.sqrt(mean_squared_error(np.exp(y), np.exp(en.predict(X))))
pd.DataFrame({'model':['cat', 'xg', 'lasso'],'coef':en.final_estimator_.coef_})

,model,coef
0,cat,0.553121
1,xg,0.363906
2,lasso,0.141271


In [ ]:
# Try 2 seperate ensembles

In [28]:
mars_pred = earth.predict(data)
mars_pred_resid = earth2.predict(data)
mars_pred_resid = np.where(mars_pred_resid < -0.6, -0.6, mars_pred_resid)

final_mars_pred = mars_pred - mars_pred_resid

In [31]:
mars_pred_resid.max()

# (np.exp(final_mars_pred) > 100).sum()

0.040272197873079923

In [27]:
a = earth2.predict(X)
a.max()

0.3169378706690221

In [13]:
from sklearn.linear_model import LinearRegression
preds = pd.DataFrame()
preds['lasso'] = lasso.predict(X)
preds['cat'] = model_cat.predict(X)
# preds['ensemble'] = en.predict(X)
preds['mars'] = final_pred
# preds['nn'] = nn.predict(X)

linear = LinearRegression()
linear.fit(preds, y)


test_preds = pd.DataFrame()
test_preds['lasso'] = lasso.predict(data)
test_preds['cat'] = model_cat.predict(data)

earth_preds = earth.predict(data)
earth_residuals = earth2.predict(data)
final_earth_preds = earth_preds - earth_residuals
test_preds['mars'] = final_earth_preds

final_preds = linear.predict(test_preds)
final_preds
# total_ensemble_preds = linear.predict(X)


NameError: ignored

In [ ]:
max(earth_residuals)

0.03722828953467671

In [78]:
# Test Predictions
data = pd.read_csv('test.csv')
id = data.id

data = data.drop(nan_col, axis=1)

imputer = impute.KNNImputer(n_neighbors=5)
data = imputer.fit_transform(data)

data = scaler.transform(data)

data = pd.DataFrame(data)

data.columns = data.columns.astype(int)
data = data.drop(res, axis=1)
# data = data.drop(columns=drops, axis=1)
data.columns = data.columns.astype(str)

# selected_predictors.remove('random')
data = data.loc[:,selected_predictors]


print(data.shape)

# test_pred = np.exp(en.predict(data))
test_pred = np.exp(en.predict(data))
# print(test_pred)

# test_pred = (test_pred + nn_pred)
# test_pred = np.exp(nn.predict(data)) * 0.2 + np.exp(model_cat.predict(data)).reshape(4403, 1) * 0.8
# test_resid_pred = earth2.predict(data)

# final_test_pred = test_pred - test_resid_pred

(4403, 625)


In [40]:
preds = pd.DataFrame()
preds['ens'] = en.predict(X)
# preds['nn'] = nn.predict(X)
preds['mars'] = earth_train_pred

linear = LinearRegression()
linear.fit(preds, y)

print(np.sqrt(mean_squared_error(np.exp(y), np.exp(linear.predict(preds)))))

test_preds = pd.DataFrame()
test_preds['ens'] = en.predict(data)
# test_preds['nn'] = nn.predict(data)
test_preds['mars'] = final_mars_pred


test_pred = np.exp(linear.predict(test_preds))


5.30206688688853


In [45]:
#MARS only
# test_pred = np.exp(final_mars_pred)

In [79]:
data = pd.read_csv('test.csv')
id = data.id

output = pd.DataFrame()
output['id'] = id
output['y'] = test_pred
# output['y'] = np.exp(final_preds)


output.loc[output['y']<0, 'y'] = 0
output.loc[output['y']>100, 'y'] = 100
output

,id,y
0,5380,3.784065
1,5381,6.167763
2,5382,2.742990
3,5383,5.580075
4,5384,2.386878
...,...,...
4398,9778,2.538770
4399,9779,8.266107
4400,9780,7.993801
4401,9781,9.512733


In [ ]:
np.exp(model_cat.predict(data).reshape(4403, 1))

array([[3.93653963],
       [6.42532094],
       [2.83547529],
       ...,
       [6.52425269],
       [8.8171908 ],
       [7.26692295]])

In [80]:
output = output.set_index('id')
# output.to_csv('Submissions/submission28.csv')
output.to_csv('submission65.csv')

In [ ]:
# selected_predictors

In [ ]:
!git clone --single-branch --branch v0.2dev https://github.com/scikit-learn-contrib/py-earth.git
%cd py-earth
!python setup.py install --cythonize

Cloning into 'py-earth'...
remote: Enumerating objects: 3303, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 3303 (delta 21), reused 17 (delta 17), pack-reused 3278
Receiving objects: 100% (3303/3303), 13.03 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
/content/py-earth
Compiling pyearth/_basis.pyx because it changed.
Compiling pyearth/_record.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_pruning.pyx because it changed.
Compiling pyearth/_forward.pyx because it depends on ./pyearth/_util.pxd.
Compiling pyearth/_knot_search.pyx because it changed.
Compiling pyearth/_qr.pyx because it depends on ./pyearth/_types.pxd.
[1/6] Cythonizing pyearth/_basis.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/py-earth/pyearth/_basis.pxd
  

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 16.8 MB/s eta 0:00:00
